<a href="https://colab.research.google.com/github/dandaramcsousa/data-analysis/blob/master/Emicinos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**código desenvolvido por [Dandara Sousa](https://dandaramcsousa.github.io) 2020.**


Metas:

* Coletar todas músicas e álbuns de Emicida
* Coletar todos os feats de Emicida
* Identificar o álbum mais popular  
* Identificar o single mais popular
* Qual é a maior música
* Qual a menor música
* Qual a música com mais feat
* Linha do tempo de lançamentos


In [9]:
#dependencias e credenciais
!pip install spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import sys
import csv

sp = spotipy.Spotify(client_credentials_manager= SpotifyClientCredentials(client_id="##", client_secret="##"))

In [2]:
'''
 dado o id/url/urn de um artista, retorna o id 
 sp.artist(url/uri/id) retorna um objeto artista que contém as infos de um artista/banda
 nesse caso queremos apenas o id desse objeto
''' 
def get_artist_id(url):
    return sp.artist(url)['id']

In [3]:
# dado o id do artista, retorna todos os álbuns que o artista fez ou colaborou
def get_artist_albums(artist_id):
    albums = []
    results = sp.artist_albums(artist_id, country = 'BR')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    return albums

In [4]:
# dada uma lista de álbuns, retorna um dicionário id: name com os álbuns autorais do artista
def get_authorial_albums(albums):
    albums_id_name = {}
    for album in albums:
        if album['album_group'] != 'appears_on':
            albums_id_name[album['id']] = album['name']
    return albums_id_name
            

In [5]:
'''
 dado o id do álbum, retorna todas as músicas de um álbum de um artista 
 gera um dicionário com as seguintes informações das músicas do álbum: álbum, 
 número da música no álbum, id, nome, popularidade, duração, 
 data e artistas envolvidos
''' 
def get_album_tracks(album_id):
    spotify_album = {}
    album = sp.album(album_id)
    
    results = sp.album_tracks(album_id) # para cada álbum, retorna as informações das músicas
    tracks = results['items']
  
    for track in tracks:
        track_id = track['id']
        track_ = sp.track(track_id) # essa função tem mais info da faixa que acessando pelo álbum

        spotify_album[track_id] = {}
    
        spotify_album[track_id]['album'] = track_['album']['name']
        spotify_album[track_id]['album_type'] = album['album_type']
    
        spotify_album[track_id]['track_id'] = track_id
        spotify_album[track_id]['track_number'] = track_['track_number']
        
        spotify_album[track_id]['name'] = track_['name']
        spotify_album[track_id]['popularity'] = track_['popularity']
        spotify_album[track_id]['duration_ms'] = track_['duration_ms']
        spotify_album[track_id]['release_date'] = track_['album']['release_date']

        track_artists = track_['artists']
        
        spotify_album[track_id]['artists'] = []
        for artist in track_artists:
            spotify_album[track_id]['artists'].append(artist['name'])
        
    return spotify_album

In [6]:
'''
 dado um dicionario com ids e nomes de álbuns (provenientes de get_authorial_albums), 
 retorna todas as músicas de todos os albuns de um artista
''' 

def get_all_albums_tracks(authorial_albums):
    spotify_albums = []
    albums_names = []
  
    for album_id, album_name in authorial_albums.items():
        print('Adicionando: ',album_id,album_name)

        if album_name not in albums_names:
            albums_names.append(album_name)
            album_tracks = get_album_tracks(album_id) 
             
        for track_infos in album_tracks.values():
            spotify_albums.append(track_infos)
            
    print('Finalizado!')
    
    return spotify_albums

In [16]:
# converte para um csv chamado 'emicinos' os dados
import pandas as pd

def convert_to_csv(filepath, name):
    df = pd.DataFrame(filepath)
    df.to_csv('../data/'+name+'.csv')
    return df.head()

**O que temos aqui?**

Ao fim dessas funções temos a possibilidade de conseguir todas as músicas de um artista e colocá-las num .csv.

In [17]:
def main():
    emicida = 'https://open.spotify.com/artist/2d9LRvQJnAXRijqIJDDs2K'
    if emicida:
        artist_id = get_artist_id(emicida)
    
        albums = get_artist_albums(artist_id)
        authorial_albums = get_authorial_albums(albums)
        #amarelo = get_album_tracks('4cT2oNoqRan9HJiICEU9Ai')
        #print(amarelo)
    
        all_albums = get_all_albums_tracks(authorial_albums)
        convert_to_csv(all_albums, 'emicinos')
    else:
        logger.error("Can't find artist: %s", artist)


if __name__ == '__main__':
    main()

Adicionando:  5cUY5chmS86cdonhoFdn8h AmarElo
Adicionando:  6nFULR21EyYSzxPUr60S3a 10 Anos de Triunfo (Ao Vivo)
Adicionando:  77ye4kGcWBmzcLWFiSCljE Sobre Crianças, Quadris, Pesadelos e Lições de Casa...
Adicionando:  4UqGzHq5gbqDhFI27IPVd2 O Glorioso Retorno de Quem Nunca Esteve Aqui
Adicionando:  4ZLLVCn6RnlENQ0nhAS5sS Rock In Rio (Ao Vivo)
Adicionando:  3xfZtthn2dlFtFfyyYKEf8 Criolo E Emicida Ao Vivo (Live)
Adicionando:  57PWjWHzqzODblomXxnQca O Glorioso Retorno de Quem Nunca Esteve Aqui
Adicionando:  24FRiS4aTYXDcL48UPknP7 Doozicabraba e a Revolução Silenciosa
Adicionando:  1xTiVYWuLx7GkZN9NLRVEs Emicidio
Adicionando:  3ZA85zeudxFA1NGeb6avXS Pra Quem Já Mordeu um Cachorro por Comida Até Que Eu Cheguei Longe
Adicionando:  1H8KrDDFycL7pXqVO5St32 Mulheres Não Têm Que Chorar
Adicionando:  5AYV4MJslPN48xaab39ZuW Trevo, Figuinha e Suor na Camisa
Adicionando:  29Kl0YXBxv72UGgM8McFv0 Sementes
Adicionando:  3yCfk5tZsvi3cWfLPzsqf3 Quem Tem Um Amigo (Tem Tudo)
Adicionando:  0VkPkc6tXvv5x8e9N4V